In [4]:
# Robot class

from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')      
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [6]:
# Make a robot called myrobot
myrobot = robot()
# that starts at coordinates 30, 50 heading north (pi/2).
myrobot.set(30.0,50.0,pi/2)
# Have your robot turn clockwise by pi/2, move 15 m.
myrobot.move(-pi/2,15.0)
# and sense.
print(myrobot.sense())
# Then have it turn clockwise by pi/2 again, move 10 m, and sense again.
myrobot.move(-pi/2, 10.0)
print(myrobot.sense())

[31.622776601683793, 58.309518948453004, 31.622776601683793, 58.309518948453004]
[31.622776601683793, 58.309518948453004, 31.622776601683793, 58.309518948453004]


In [7]:
# Make a robot called myrobot
myrobot = robot()
# forward_noise = 5.0, turn_noise = 0.1, sense_noise = 5.0.
myrobot.set_noise(5.0,0.1,5.0)
myrobot.set(30.0, 50.0, pi/2)
# that starts at coordinates 30, 50 heading north (pi/2).
myrobot.set(30.0,50.0,pi/2)
# Have your robot turn clockwise by pi/2, move 15 m.
myrobot.move(-pi/2,15.0)
# and sense.
print(myrobot.sense())
# Then have it turn clockwise by pi/2 again, move 10 m, and sense again.
myrobot.move(-pi/2, 10.0)
print(myrobot.sense())

[31.79596568641487, 56.43888846103205, 40.90517516226268, 69.56639743705013]
[41.490046205077604, 59.099253406944776, 42.780406075588054, 60.9294805605084]


### Create 1000 random particles

In [9]:
N=1000
p = [robot() for i in range(N)]
print(len(p))

1000


### Each particle should turn by 0.1 and then move by 5.

In [11]:
# Each particle should turn by 0.1 and then move by 5.
p = [robot.move(0.1,5) for robot in p]

## Importance weights
- how important the particle is
- it will be set based on the mismatch between predicted distance to landmarks and actual measured distance to landmarks.

In [25]:
# actual robot measurements
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
# 1000 random particles with noises
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

# Each particle should turn by 0.1 and then move by 5.
p = [particle.move(0.1,5) for particle in p]

# set 1000 particle weights.
w = [x.measurement_prob(Z) for x in p]

## Resampling
- draw more frequently if importance weight is higher
- here is an example


|particle|weight|normalization|
|---|---|---|
|p1|w1 = 0.6|alpha=0.1|
|p2|w2 = 1.2|alpha=0.2|

In [26]:
p3 = []
index = int(random.random() * N)
beta = 0.0
mw = max(w)
for i in range(N):
    beta += random.random() * 2.0 * mw
    while beta > w[index]:
        beta -= w[index]
        index = (index + 1) % N
    p3.append(p[index])
p = p3

In [27]:
p

[[x=37.262 y=36.111 orient=0.0262],
 [x=45.587 y=39.980 orient=3.6167],
 [x=42.439 y=42.028 orient=5.8318],
 [x=37.759 y=37.785 orient=1.1632],
 [x=38.167 y=40.115 orient=0.6841],
 [x=38.167 y=40.115 orient=0.6841],
 [x=41.484 y=33.506 orient=4.5059],
 [x=37.605 y=45.852 orient=5.4680],
 [x=36.069 y=41.785 orient=4.9978],
 [x=37.262 y=36.111 orient=0.0262],
 [x=41.362 y=39.751 orient=5.6048],
 [x=44.271 y=39.459 orient=3.4365],
 [x=38.167 y=40.115 orient=0.6841],
 [x=41.609 y=37.266 orient=5.5655],
 [x=41.609 y=37.266 orient=5.5655],
 [x=39.928 y=40.124 orient=0.3200],
 [x=39.928 y=40.124 orient=0.3200],
 [x=37.605 y=45.852 orient=5.4680],
 [x=41.343 y=46.453 orient=5.9877],
 [x=36.069 y=41.785 orient=4.9978],
 [x=42.051 y=43.590 orient=5.4680],
 [x=38.705 y=48.940 orient=0.1131],
 [x=41.362 y=39.751 orient=5.6048],
 [x=44.271 y=39.459 orient=3.4365],
 [x=37.759 y=37.785 orient=1.1632],
 [x=38.167 y=40.115 orient=0.6841],
 [x=41.609 y=37.266 orient=5.5655],
 [x=39.928 y=40.124 orient=0